<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week04_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 4



## Shallow Neural Networks



### 7.1 Neural Networks in One Dimension



In [ ]:
import os


import matplotlib.pyplot as plt
import torch
import torch.nn as nn



In [ ]:
torch.manual_seed(0)


class Net(torch.nn.Module):
  def __init__(self, *argv, **kwarg):
    super(Net, self).__init__()

    self.linears = torch.nn.ModuleList([])
    for n_in, n_out in (argv[:-1], argv[1:]):
      self.linears.append(torch.nn.Linear(n_in, n_out))

  def forward(self, x):

    for layer in self.linears:
      x = torch.sigmoid(layer(x))

    return x

  def plot_activation(self, Y, X):
      a1 = torch.sigmoid(self.linears[0](X))
      plt.scatter(
          a1.detach().numpy()[:, 0],
          a1.detach().numpy()[:, 1],
          c=Y.numpy().reshape(-1)
      )
      plt.title('activations')
      plt.grid(True)


# https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html
# https://stackoverflow.com/questions/50463975/pytorch-how-to-properly-create-a-list-of-nn-linear
# https://discuss.pytorch.org/t/when-should-i-use-nn-modulelist-and-when-should-i-use-nn-sequential/5463



* 1 input node
* two nodes in the input layer
* one node on the output layer



$$
\begin{align}
z_{N \times 2}^1&=x_{N \times 1}^1W_{1 \times 2}^1+b_{N \times 2}^1 \\
x_{N \times 2}^2&=\sigma(z_{N \times 2}^1) \\
z_{N \times 1}^2&=x_{N \times 2}^2W_{2 \times 1}^2+b_{N \times 1}^2 \\
y_{N \times 1}&=\sigma(z_{N \times 1}^2)
\end{align}
$$



In [ ]:
model = Net(1, 2, 1)

x = torch.tensor([0.0])

yhat = model(x)
yhat



In [ ]:
model.state_dict()



Sequential version



In [ ]:
torch.manual_seed(1)

model = torch.nn.Sequential(
    torch.nn.Linear(1, 2),
    torch.nn.Sigmoid(),
    torch.nn.Linear(2, 1),
    torch.nn.Sigmoid(),
)

x = torch.tensor([0.0])

yhat = model(x)

yhat



In [ ]:
x = torch.tensor([
    [0.0],
    [1.0],
    [2.0],
  ])

yhat = model(x)

yhat



In [ ]:
model.state_dict()



Train the model



In [ ]:
X = torch.linspace(-20, 20, 41).view(-1, 1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0])
Y[
    (X[:, 0] > -4.0) & (X[:, 0] < 4.0)
] = 1.0



In [ ]:
def plot_y_yhat(X, Y, model, epoch, legend=True):
    plt.plot(
        X.numpy(), model(X).detach().numpy(),
        label=('epoch '+str(epoch))
    )
    plt.plot(X.numpy(), Y.numpy(), 'r')
    plt.xlabel('x')
    if legend:
      plt.legend(loc=0)
    plt.grid(True)



In [ ]:
def train(Y, X, model, optimizer, criterion, n_epoch=1000):
  cost = []
  total = 0

  if os.environ.get('CI', False):
    n_epoch = 1

  for epoch in range(n_epoch):
    total = 0.0

    for y, x in zip(Y, X):
      yhat = model(x)

      y = y.unsqueeze(-1)
      # https://stackoverflow.com/questions/57798033/valueerror-target-size-torch-size16-must-be-the-same-as-input-size-torch
      loss = criterion(yhat, y)
      loss.backward()

      optimizer.step()
      optimizer.zero_grad()

      total += loss.item()

    cost.append(total)

  return cost



In [ ]:
model = Net(1, 2, 1)

plot_y_yhat(X, Y, model, 0, legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
loss_list = train(
    Y=Y, X=X, model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.1),
    criterion=torch.nn.BCELoss(),
    n_epoch=1 # use 1000 to train
)



In [ ]:
plot_y_yhat(X, Y, model, len(loss_list), legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
plt.plot(loss_list)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid(True)



### 7.2 Neural Networks More Hidden Neurons



More complicated data



In [ ]:
X = torch.linspace(-20, 20, 41).view(-1, 1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0])
Y[
    ((X[:, 0] > -10.0) & (X[:, 0] < -5.0))
    | ((X[:, 0] > 5.0) & (X[:, 0] < 10.0))
] = 1.0



In [ ]:
model = Net(1, 2, 1)

plot_y_yhat(X, Y, model, 0, legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
loss_list = train(
    Y=Y, X=X, model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.1),
    criterion=torch.nn.BCELoss(),
    n_epoch=1000,
)



In [ ]:
plot_y_yhat(X, Y, model, len(loss_list), legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
plt.plot(loss_list)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid(True)



### 7.3 Neural Networks with Multiple Dimensional Input



### 7.4 Multi-Class Neural Networks



### 7.5 Backpropagation



### 7.6 Activation Function

